In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-12-03 22:19:59--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-03 22:19:59--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-03 22:20:00--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [26]:
import pickle
input_df_train, input_df_test, train_y, test_y = pickle.load(open('/content/drive/MyDrive/bdp_project_final_train_test.p', mode='rb'))

In [27]:
for col in ['rating', 'duration_long','n_posts_by_author']:
    input_df_train.drop(col, axis=1, inplace=True)
    input_df_test.drop(col, axis=1, inplace=True)

In [28]:
input_df_train.fillna(False, inplace=True)
input_df_test.fillna(False, inplace=True)

In [29]:
def normalize(X):
    for col in ['n_keywords', 'n_posts_by_channel']:
        X[col] = (X[col]-X[col].mean() )/X[col].std()

    return X

In [30]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

def score(test_y, ypred_test_final):
    print(confusion_matrix(test_y, ypred_test_final))
    print(classification_report(test_y, ypred_test_final))
    print('accuracy_score:  '+str(accuracy_score(test_y, ypred_test_final)))
    print('f1_score:        '+str(f1_score(test_y, ypred_test_final)))
    print('precision_score: '+str(precision_score(test_y, ypred_test_final)))
    print('recall_score:    '+str(recall_score(test_y, ypred_test_final)))

In [31]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

lmod = LogisticRegression()
lmod.fit(normalize(input_df_train), train_y)
ypred_test_final = lmod.predict(normalize(input_df_test))

In [32]:
score(test_y, ypred_test_final)
coefs = pd.DataFrame(lmod.coef_).T
coefs.index = input_df_train.columns
coefs.columns=['coef']
coefs.sort_values('coef')

[[6832 2303]
 [3377 5357]]
              precision    recall  f1-score   support

           0       0.67      0.75      0.71      9135
           1       0.70      0.61      0.65      8734

    accuracy                           0.68     17869
   macro avg       0.68      0.68      0.68     17869
weighted avg       0.68      0.68      0.68     17869

accuracy_score:  0.682131064972858
f1_score:        0.6535317799194827
precision_score: 0.6993472584856397
recall_score:    0.6133501259445844


,coef
duration_short,-0.136131
on_trend_transcript,-0.013751
n_posts_by_channel,-0.000948
transcript_prediction,0.000000
title_prediction,0.000000
description_prediction,0.000000
post_covid,0.000000
has_sounds,0.214480
on_trend_description,0.258432
duration_med,0.382834


In [33]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(input_df_train, train_y)
ypred_test_final = rf.predict(input_df_test)


In [34]:
score(test_y, ypred_test_final)
feature_importance = pd.DataFrame(rf.feature_importances_)
feature_importance.index=input_df_train.columns
feature_importance.columns=['feature_importance']
feature_importance.sort_values('feature_importance', ascending=False)

[[7032 2103]
 [2950 5784]]
              precision    recall  f1-score   support

           0       0.70      0.77      0.74      9135
           1       0.73      0.66      0.70      8734

    accuracy                           0.72     17869
   macro avg       0.72      0.72      0.72     17869
weighted avg       0.72      0.72      0.72     17869

accuracy_score:  0.717219766075326
f1_score:        0.6959870043920341
precision_score: 0.7333586915176873
recall_score:    0.6622395237004809


,feature_importance
n_keywords,0.464228
n_posts_by_channel,0.430479
has_music,0.024741
duration_med,0.021787
duration_short,0.020966
has_sounds,0.017063
on_trend_transcript,0.008316
on_trend_description,0.005991
age_restricted,0.004149
on_trend_title,0.002280


In [35]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')
svc.fit(normalize(input_df_train), train_y)
ypred_test_final = svc.predict(normalize(input_df_test))

In [36]:
score(test_y, ypred_test_final)
svc_coefs = pd.DataFrame(svc.coef_).T
svc_coefs.index = input_df_train.columns
svc_coefs.columns=['coef']
svc_coefs.sort_values('coef',ascending=False)

[[6819 2316]
 [3298 5436]]
              precision    recall  f1-score   support

           0       0.67      0.75      0.71      9135
           1       0.70      0.62      0.66      8734

    accuracy                           0.69     17869
   macro avg       0.69      0.68      0.68     17869
weighted avg       0.69      0.69      0.68     17869

accuracy_score:  0.6858246124573283
f1_score:        0.6594686400582312
precision_score: 0.7012383900928792
recall_score:    0.6223952370048088


,coef
age_restricted,1.154015
n_keywords,0.926307
has_music,0.614547
on_trend_title,0.384656
duration_med,0.307864
on_trend_description,0.307251
has_sounds,0.231140
transcript_prediction,0.000000
title_prediction,0.000000
description_prediction,0.000000
